In [ ]:
Pkg.add("Gadfly")
using Gadfly
using DataFrames

# Parameters

In [ ]:
@enum DataSource concentriccircles=1 spiral=2

samples_per_class = 100; # number of data points per class
data_distortion = 0.02; # set to 0 for no distortion
data_source = spiral; # spiral or circle
num_classes = 3; # number of different classes

const num_dimensions = 2; # Number of dimensions (x, y), changing this will break all plotting

# computed values
N = samples_per_class * num_classes

# Generate our data sets

In [ ]:
# generate data
function getCircle(dots, radius, x=0, y=0)
    angles = randn(dots).*(2*pi)
    xs = cos(angles).*radius.+x
    ys = sin(angles).*radius.+y
    return [ xs ys ]
end

function getSpiral(dots, degrees, rate=1.0)
    angles = linspace(pi/3, degrees, dots)
    radii = angles.*rate
    X = [ f(angles[i]) * radii[i] for i in 1:dots, f in [cos, sin]]
end

if data_source == concentriccircles
    Data = getCircle(samples_per_class, 1.5)
    Labels = ones(Int64, samples_per_class)
    for i in 2:num_classes
        Data = vcat(Data, getCircle(samples_per_class, i*1.5, 2, 2))
        Labels = vcat(Labels, ones(Int64, samples_per_class).*i)
    end
elseif data_source == spiral
    Data = getSpiral(samples_per_class, 2pi, 1.5)
    Labels = ones(Int64, samples_per_class)
    for i in 2:num_classes
        Data = vcat(Data, getSpiral(samples_per_class, 2pi, i*1.5))
        Labels = vcat(Labels, ones(Int64, samples_per_class).*i)
    end
end

In [ ]:
# Visualization helper functions
function predictClasses(data, getScores)
    scores = getScores(data)
    exp_scores = e.^scores
    _, max_indices = findmax(exp_scores', 1)
    transpose((max_indices-1)%num_classes+1)
end

function output_learning_status_image(iteration, getScores)
    data = [ [x y] for x in linspace(-1, 1, 50), y in linspace(-1, 1, 50) ]
    data = vcat(data...)
    data = convert(DataFrame, [data predictClasses(data, getScores)])
    original_data = convert(DataFrame, [DistortedData Labels])
    p = plot(
        layer(original_data, x=:x1, y=:x2, color=:x3, Geom.point),
        layer(data, x=:x1, y=:x2, color=:x3, Geom.point),
        Guide.title("Iteration $(iteration)")
    )
    draw(SVGJS("./images/test_$(lpad(iteration,6,0)).js.svg", 6inch, 6inch), p)
end

function plot_data(df)
    plot(df, x=:x1, y=:x2, color=:x3, Geom.point)
end

In [ ]:
# Normalize the data so that it is between -1 and 1
NormalizedData = Data ./ repmat(maximum(abs(Data), 1), N) 
plot_data(convert(DataFrame, [NormalizedData Labels]))

In [ ]:
# Add some randomization to the dataset
DistortedData = NormalizedData + randn(N, num_dimensions).*data_distortion;
plot_data(convert(DataFrame, [DistortedData Labels]))

# Implement linear regression

In [ ]:
# Basic linear regression implementation
function linear_regression(data, labels)
    
    # parameters
    step_size = 1e-1;
    iterations = 1000;
    regularization = 1e-3; # regularization strength
    
    # randomly initialize weights
    W = randn(num_dimensions, num_classes) .* 0.01

    # randomly initialize bias vector
    b = zeros(1, num_classes)
    
    # compute the gradient on normalized_scores (based on the derivative of the loss function)
    correct_class_bitmask = zeros(N, num_classes)
    for (index, label_idx) in enumerate(labels)
        correct_class_bitmask[index, Int64(label_idx)] = 1
    end

    # begine linear regression
    for iter in 1:iterations

        # compute scores
        scores = data * W + repmat(b, N)

        # compute normalized log probabilities (e^scores)
        exp_scores = exp(scores)

        # e^scores / ( sum(e^scores) )
        normalized_scores = exp_scores ./ repmat(sum(exp_scores, 2)', size(exp_scores, 2))'

        # normalized log-likelyhood -log(e^scores / ( sum(e^scores)))
        normalized_log_likelyhood = -log(normalized_scores[map(b -> b == 1 ,correct_class_bitmask)])
        total_error = sum(normalized_log_likelyhood)/N + 0.5*regularization*sum(W.*W)
        if iter < 10 || iter % 100 == 0
            println("ITER: $(iter), ERROR: $(total_error)")
            output_learning_status_image(iter, (d) -> d * W + repmat(b, size(d, 1)))
        end
        
        # log likelihood gradient
        dscores = normalized_scores - correct_class_bitmask
        dscores ./= N
        
        dW = data'*dscores + regularization.*W 
        db = sum(dscores, 1)
        
        W += -step_size .* dW
        b += -step_size * db
    end
    return W, b
end

In [ ]:
linear_regression(DistortedData, Labels)

# 2 Layer Feed Forward Neural Network

In [ ]:
function linear_regression_two_layers(data, labels)
    step_size = 1e-1
    iterations = 10000
    hidden_layer_size = 100
    regularization = 1e-3; # regularization strength
        
    # randomly initialize weights
    W = randn(num_dimensions, hidden_layer_size) .* 0.01
    W2 = randn(hidden_layer_size, num_classes).* 0.01
    
    # randomly initialize bias vector
    b = zeros(1, hidden_layer_size)
    b2 = zeros(1, num_classes)
    
    # compute the gradient on normalized_scores
    correct_class_bitmask = zeros(N, num_classes)
    for (index, label_idx) in enumerate(labels)
        correct_class_bitmask[index, Int64(label_idx)] = 1
    end

    # begine linear regression
    for iter in 1:iterations

        # compute scores (forward pass)
        hidden_layer = max(0, data * W + repmat(b, N))
        scores = hidden_layer * W2 + repmat(b2, N)

        # compute normalized log probabilities (e^scores)
        exp_scores = e.^scores

        # e^scores / ( sum(e^scores) )
        normalized_scores = exp_scores ./ repmat(sum(exp_scores, 2)', size(exp_scores, 2))'

        # normalized log-likelyhood -log(e^scores / ( sum(e^scores)))
        normalized_log_likelyhood = -log(normalized_scores[map(b -> b == 1, correct_class_bitmask)])
        total_error = sum(normalized_log_likelyhood)/N 
            + 0.5*regularization*sum(W.*W) 
            + 0.5*regularization*sum(W2.*W2)
        if (iter < 100 && iter % 10 == 0) || iter % 1000 == 0
            println("ITER: $(iter), ERROR: $(total_error)")
            output_learning_status_image(
                iter, 
                (d) -> max(0, d*W+repmat(b, size(d, 1)))*W2+repmat(b2, size(d, 1))
            )
        end
            
        # log likelihood gradient
        dscores = normalized_scores - correct_class_bitmask
        dscores ./= N
        
        # backpropate the gradient to the parameters
        # first backprop into parameters W2 and b2
        dW2 = hidden_layer'*dscores + regularization .* W2
        db2 = sum(dscores, 1)
        
        # next backprop into hidden layer
        dhidden = dscores*W2'
        
        # backprop the ReLU non-linearity
        dhidden[hidden_layer .<= 0] = 0

        # finally into W,b
        dW = data' * dhidden + regularization .* W
        db = sum(dhidden, 1)
        
        # update our parameters
        W += -step_size .* dW
        b += -step_size * db
        W2 += -step_size .* dW2
        b2 += -step_size * db2
    end
end

In [ ]:
linear_regression_two_layers(DistortedData, Labels)